In [ ]:
import analyze_results_code
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shap
from sklearn.model_selection import train_test_split
import pickle
from pathlib import Path
import os

!pip install jupyter-black --quiet

%load_ext jupyter_black

# GLOBAL ANALYSIS

## Load ranked contingencies

In [ ]:
# df_contg = analyze_results_code.load_df("/home/guiu/Projects/CONT_SCR_CRV_REC/Data")
def load_df(path):
    path = Path(path)
    first_df = True
    df_contg = pd.DataFrame()  # Inicializar un DataFrame vacío

    for year_dir in path.iterdir():
        if year_dir.is_file():
            continue
        for month_dir in year_dir.iterdir():
            if month_dir.is_file():
                continue
            for day_dir in month_dir.iterdir():
                if day_dir.is_file():
                    continue
                for time_dir in day_dir.iterdir():
                    if os.path.isfile(time_dir):
                        if first_df:
                            first_df = False
                            df_contg = pd.read_csv(time_dir, sep=";")
                            time_str = str(time_dir.stem)[-22:-9]
                            df_contg["DATE"] = time_str + "00"
                            df_contg["DATE"] = pd.to_datetime(
                                df_contg["DATE"], format="%Y%m%d_%H%M%S"
                            )
                        else:
                            df_new = pd.read_csv(time_dir, sep=";")
                            time_str = str(time_dir.stem)[-22:-9]
                            df_new["DATE"] = time_str + "00"
                            df_new["DATE"] = pd.to_datetime(df_new["DATE"], format="%Y%m%d_%H%M%S")
                            df_contg = pd.concat([df_contg, df_new], axis=0, ignore_index=False)

    return df_contg.dropna()


df_contg = load_df("/home/guiu/Projects/CONT_SCR_CRV_REC/Data/Results_Hades_19_03")
df_contg["PREDICTED_SCORE"] = pickle.load(
    open("/home/guiu/Projects/CONT_SCR_CRV_REC/Data/Models_19_03/GBR_model.pkl", "rb")
).predict(df_contg.drop(columns=["PREDICTED_SCORE", "STATUS", "REAL_SCORE", "DATE", "NAME"]))
df_contg = df_contg.set_index("NAME")
df_contg = df_contg.reindex(
    [
        "DATE",
        "STATUS",
        "PREDICTED_SCORE",
        "REAL_SCORE",
        "MIN_VOLT",
        "MAX_VOLT",
        "MAX_FLOW",
        "N_ITER",
        "AFFECTED_ELEM",
        "CONSTR_GEN_Q",
        "CONSTR_GEN_U",
        "CONSTR_VOLT",
        "CONSTR_FLOW",
        "RES_NODE",
        "COEF_REPORT",
        "TAP_CHANGERS",
    ],
    axis=1,
)

## Show DynaFlow simulation failures
Show all contingencies where DynaFlow has failed for the set of simulations provided.

In [ ]:
df_contg[df_contg["STATUS"] == "HDS"].sort_values("N_ITER", ascending=False)

In [ ]:
df_contg[df_contg["STATUS"] == "HDS"].index.value_counts()

## Show Hades non-convergence
Show all contingencies where Hades has failed for the set of simulations provided.

In [ ]:
df_contg[df_contg["STATUS"] == "DWO"]

In [ ]:
df_contg[df_contg["STATUS"] == "DWO"].index.value_counts()

## Worst contingencies (For cases where both converged)

In [ ]:
df_contg = df_contg.sort_values(by="REAL_SCORE", ascending=False)
df_contg[df_contg["STATUS"] == "BOTH"]

## MAE (Real score vs Predicted score)

In [ ]:
df_filtered = df_contg[df_contg["STATUS"] == "BOTH"]

mae = mean_absolute_error(df_filtered["REAL_SCORE"], df_filtered["PREDICTED_SCORE"])
print("Mean Absolute Error:", mae)

## All-time top N most different contingencies (using the Real score, median-based statistic)

In [ ]:
analyze_results_code.all_time_top(df_filtered)

## Week day top N most different contingencies (using the Real score, median-based statistic)

In [ ]:
analyze_results_code.week_day_top(df_filtered)

## Month top N most different contingencies (using the Real score, median-based statistic)

In [ ]:
analyze_results_code.month_top(df_filtered)

## Hour top N most different contingencies (using the Real score, median-based statistic)

In [ ]:
analyze_results_code.hour_top(df_filtered)

## Calculate quantiles

In [ ]:
for i in range(0, 100, 5):
    print(i / 100, df_contg["REAL_SCORE"].quantile(i / 100))

## Hour boxplot of real scores

In [ ]:
analyze_results_code.hour_boxplot(df_contg, "REAL_SCORE")

## Hour boxplot of predicted scores

In [ ]:
analyze_results_code.hour_boxplot(df_contg, "PREDICTED_SCORE")

## Day boxplot of real scores

In [ ]:
analyze_results_code.day_boxplot(df_contg, "REAL_SCORE")

## Day boxplot of predicted scores

In [ ]:
analyze_results_code.day_boxplot(df_contg, "PREDICTED_SCORE")

## Real score histogram

In [ ]:
analyze_results_code.score_histogram(df_contg, "REAL_SCORE")

## Predicted score histogram

In [ ]:
analyze_results_code.score_histogram(df_contg, "PREDICTED_SCORE")

## Shap values

In [ ]:
# Get shap values
np.bool = bool
explainer = shap.TreeExplainer(
    pickle.load(
        open("/home/guiu/Projects/CONT_SCR_CRV_REC/Data/Models_19_03/GBR_model.pkl", "rb")
    ),
    link="logit",
)

In [ ]:
_, X_test = train_test_split(
    df_contg.drop(columns=["PREDICTED_SCORE", "STATUS", "REAL_SCORE", "DATE"]),
    test_size=0.2,
    random_state=42,
)


shap_values = explainer.shap_values(X_test)

plt.clf()
shap.summary_plot(shap_values, X_test, show=False)
plt.show()

In [ ]:
shap.dependence_plot(
    "MIN_VOLT",
    shap_values,
    X_test,
)

In [ ]:
shap.dependence_plot(
    "MAX_VOLT",
    shap_values,
    X_test,
)

In [ ]:
shap.dependence_plot(
    "MAX_FLOW",
    shap_values,
    X_test,
)

In [ ]:
shap.dependence_plot(
    "N_ITER",
    shap_values,
    X_test,
)

In [ ]:
shap.dependence_plot(
    "AFFECTED_ELEM",
    shap_values,
    X_test,
)

In [ ]:
shap.dependence_plot(
    "CONSTR_GEN_Q",
    shap_values,
    X_test,
)

In [ ]:
shap.dependence_plot(
    "CONSTR_VOLT",
    shap_values,
    X_test,
)

In [ ]:
shap.dependence_plot(
    "CONSTR_FLOW",
    shap_values,
    X_test,
)

In [ ]:
shap.dependence_plot(
    "RES_NODE",
    shap_values,
    X_test,
)

In [ ]:
shap.dependence_plot(
    "COEF_REPORT",
    shap_values,
    X_test,
)

In [ ]:
shap.dependence_plot(
    "TAP_CHANGERS",
    shap_values,
    X_test,
)